# Introduction to Coroutines in Python

Sources:
* http://www.informit.com/articles/article.aspx?p=2320938
* http://stackoverflow.com/questions/9708902/in-practice-what-are-the-main-uses-for-the-new-yield-from-syntax-in-python-3/26109157#26109157

## What is a Coroutine?

A coroutine is a generalization of a subroutine (or method). In other words: a subroutine is a *special case* of a coroutine. Contrary to a subroutine, a coroutine can be *suspended* and *resumed* (that's what the ``yield`` keyword is for).

Coroutines allow us to implement **cooperative** or **non-preemptive multitasking** (as opposed to preemptive multitasking). In cooperative multitasking, the operating system does not *force* a task/process to release the CPU (called a "context switch"), instead the tasks *voluntarily* suspend their operation periodically, allowing other tasks to run.

Here's an example:

In [8]:
def my_coroutine():
    i = 0
    while True:
        print("hello world: {}".format(i))
        yield  # coroutine stops here until it's resumed
        i += 1

Let's call ``my_coroutine`` and assign it to a variable:

In [9]:
coro = my_coroutine()
coro

<generator object my_coroutine at 0x7f3a23ffac18>

What we get is a so called "gererator" object. In modern Python, "generator" is essentially a synonym for "coroutine" (the original distinction was that generators can only *produce* data while coroutines can also *consume* it, more on this later).

Note that no code inside ``my_coroutine`` has been executed so far (otherwise we would see some output). By calling ``next()`` we can now start the coroutine. It suspends as soon as it hits a ``yield`` keyword:

In [10]:
next(coro)

hello world: 0


By executing another ``next()`` we can make it advance until it hits ``yield`` (you can think of ``yield`` as a "breakpoint"):

In [11]:
next(coro)

hello world: 1


Does it ever run out? No, you can call ``next()`` as often as you want:

In [12]:
next(coro)
next(coro)

hello world: 2
hello world: 3


## Coroutines Can Produce Data

The introductory "hello world" example is not very useful in that it does neither consume nor produce/return any data (it only ``print``s data on the screen). Let's first look at a coroutine that produces some data (we say it ``yield``s data). The statement ``yield x`` not only suspends a coroutine, but also sends an object ``x`` back to the caller (which may be another coroutine or a "normal" context).

For example, coroutines can be used to implement infinite lists. In fact, the following coroutine does not create an infinite list of natural numbers in memory (which is impossible), but *enumerates* all natural numbers:

In [18]:
def natural_numbers():
    i = 0
    while True:
        yield i  # send i back to caller
        i += 1

Note that – instead of calling ``next()`` – we can iterate through a coroutine object in the same way we would iterate through a list:

In [20]:
naturals = natural_numbers()
for i in naturals:
    print(i)
    if i > 10:  # exit condition to avoid infinite loop
        break

0
1
2
3
4
5
6
7
8
9
10
11


## Coroutines Can Consume Data

Coroutines can not only ``yield``/produce data, but data can also be sent to them using the ``send()`` method:

In [77]:
def test():
    while True:
        x = yield
        print("hello {}".format(x))

In [78]:
coro = test()

Before we can ``send`` any data, we have to advance the coroutine to the first ``yield`` statement (also called *priming* the coroutine):

In [79]:
next(coro)

Now we can send it some object. Calling ``send()`` will also iterate the coroutine (as does ``next()``):

In [80]:
coro.send("foo")
coro.send("bar")

hello foo
hello bar


If we iterate over it without sending data, ``x`` will simply be ``None``:

In [81]:
next(coro)

hello None


Now let's create a more useful coroutine. Let's implement a coroutine that ``yield``s the minimum value it's been sent so far (example taken from [here](http://www.informit.com/articles/article.aspx?p=2320938)).

In [82]:
def minimize():
    current = yield
    while True:
        value = yield current
        current = min(value, current)

In [83]:
coro = minimize()
next(coro)

In [84]:
coro.send(4)

4

In [85]:
coro.send(5)

4

In [86]:
coro.send(-2)

-2

## Coroutines versus Threads/Processes

Like threads, coroutines are independent functions that can consume input from their environment and produce resulting outputs. The difference is that coroutines pause at each ``yield`` expression and resume after each call to ``send()`` or ``next()`` from the outside. This is the magical mechanism of coroutines.

This behavior allows the code calling the coroutine to take action after each ``yield`` expression in the coroutine. The consuming code can use the generator's output values to call other functions and update data structures. Most importantly, it can advance other coroutines. By advancing many separate coroutines in lockstep, they will all seem to be running simultaneously, mimicking the concurrent behavior of threads.

Coroutines have the following advantages over threads:
* Coroutines are single-threaded. That means, no synchronization is needed to prevent race conditions, dead locks, etc.
* Threads use a lot more memory and are relatively expensive to start

Disadvantages:
* TODO

## Another example

In [ ]:
@asyncio.coroutine
def match(pattern):
    print("Looking for: " + pattern)
    try:
        while True:
            s = yield
            if pattern in s:
                print("found {} in {}".format(pattern, s))
    except GeneratorExit:
        print("Done")

In [ ]:
m = match("gugus")
m.__next__()  # executes function body until "yield" statement

In [ ]:
m.send("blagugus")  # evaluation continues inside match() until it encounters "yield"

In [ ]:
m.send("asdfsadf")

In [ ]:
m.send("gugusasdflj")

In [ ]:
m.close()

We can chain functions that ``send()`` and functions that ``yield`` together to obtain complex behaviors.

In [ ]:
@asyncio.coroutine
def read(text, coroutine):
    for word in text.split():
        coroutine.send(word)
    coroutine.close()

In [ ]:
text = "Commending spending is offending to people pending lending!"
matcher = match("ending")
matcher.__next__()

In [ ]:
read(text, matcher)

Note that ``match()`` is a "consumer" of "filter" (``yield``s, but does not ``send()``), while ``read()`` is a "producer" (produces data and sends it to another coroutine).
A producer or filter can of course have multiple coroutines downstream of it. Here is an example:

In [ ]:
@asyncio.coroutine
def read_to_many(text, coroutines):
    for word in text.split():
        for c in coroutines:
            c.send(word)
    for c in coroutines:
        c.close()

In [ ]:
m1 = match("mend")
m1.__next__()

In [ ]:
m2 = match("pe")
m2.__next__()

In [ ]:
read_to_many(text, [m1, m2])

## Coroutines and the ``asyncio`` Framework

For documentation purposes, all coroutines should be decorated with ``@asyncio.coroutine``.

TODO

## The ``yield from`` syntax

Note that ``yield from g`` is **not** equivalent to ``for v in g: yield v``. Think of ``yield from`` as providing a transparent two-way channel from the caller to the sub-coroutine. That includes both getting data from and sending data to the sub-coroutine.

Consider the following example:

In [ ]:
@asyncio.coroutine
def reader():
    """A generator that fakes a read from a file, socket, etc."""
    for i in range(4):
        yield '<< {}'.format(i)

@asyncio.coroutine
def reader_wrapper(coro):
    """Manually iterate over data produced by reader"""
    for v in coro:
        yield v

wrap = reader_wrapper(reader())
for i in wrap:
    print(i)

We can simplify ``reader_wrapper`` as follows:

In [ ]:
@asyncio.coroutine
def reader_wrapper(coro):
    yield from coro
    
wrap = reader_wrapper(reader())
for i in wrap:
    print(i)

Now let's create a coroutine that writes data to some file descriptor:

In [ ]:
@asyncio.coroutine
def writer():
    """A coroutine that writes data *sent* to it to fd, socket, etc."""
    while True:
        w = yield
        print('>> ', w)

A coroutine wrapping ``writer`` could look as follows:

In [ ]:
@asyncio.coroutine
def writer_wrapper(coro):
    coro.send(None)  # prime the coroutine (run to 1st yield statement)
    while True:
        try:
            x = yield  # Capture the value that's sent
            coro.send(x)  # and pass it to the writer
        except StopIteration:
            pass

In [ ]:
w = writer()
wrap = writer_wrapper(w)
wrap.send(None)  # "prime" the coroutine
for i in range(4):
    wrap.send(i)

However, we can simplify ``writer_wrapper`` as follows:

In [ ]:
@asyncio.coroutine
def writer_wrapper(coro):
    yield from coro
    
w = writer()
wrap = writer_wrapper(w)
wrap.send(None)  # "prime" the coroutine
for i in range(4):
    wrap.send(i)

Let's make it more complicated. What if our writer needs to handle exceptions? Let's say the writer handles a ``SpamException`` and it prints ``***`` if it encounters one.